# Enhanced ICP: Soft Correspondences with Generalized-ICP

## Introduction

In the previous pose estimation exercise, we used the Iterative Closest Point (ICP) algorithm to estimate object poses from point cloud data. While ICP works well in many scenarios, it has some limitations:

1. **Hard correspondences**: Standard ICP assumes each point in one scan corresponds exactly to its nearest neighbor in the other scan
2. **Sensitivity to incorrect matches**: Wrong correspondences can pull the alignment away from the correct solution
3. **Discretization issues**: Different sampling of the same surface makes exact point overlap impossible

In this tutorial, we'll explore **Generalized-ICP (GICP)**, a probabilistic extension that addresses these issues through *soft correspondences*. Instead of treating each match as a hard constraint, GICP models uncertainty in point positions and uses local surface structure from *both* scans.

**What you'll learn:**
- How point-to-plane ICP improves on standard ICP
- The probabilistic framework behind Generalized-ICP
- How to implement "plane-to-plane" matching with covariance matrices
- Why GICP is more robust to incorrect correspondences

**Source**: This tutorial is based on the paper "Generalized-ICP" by Segal, Haehnel, and Thrun (RSS 2009).

In [ ]:
import numpy as np
import scipy.optimize as opt
from scipy.spatial import KDTree
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    BaseField,
    DiagramBuilder,
    Fields,
    MeshcatVisualizer,
    Parser,
    PointCloud,
    Rgba,
    RigidTransform,
    RollPitchYaw,
    RotationMatrix,
    StartMeshcat,
)

from manipulation import running_as_notebook
from manipulation.scenarios import AddMultibodyTriad
from manipulation.station import AddPointClouds, LoadScenario, MakeHardwareStation

In [ ]:
# Start the visualizer
meshcat = StartMeshcat()

## Setup: Model and Scene Point Clouds

We'll use the same red foam brick example from the pose estimation exercise. This gives us a familiar setting to compare different ICP variants.

In [ ]:
def ToPointCloud(xyzs, rgbs=None):
    """Convert numpy arrays to Drake PointCloud object."""
    if rgbs is not None:
        cloud = PointCloud(xyzs.shape[1], Fields(BaseField.kXYZs | BaseField.kRGBs))
        cloud.mutable_rgbs()[:] = rgbs
    else:
        cloud = PointCloud(xyzs.shape[1])
    cloud.mutable_xyzs()[:] = xyzs
    return cloud


def generate_model_pointcloud(xrange, yrange, zrange, res):
    """Generate a point cloud of a rectangular brick."""
    x_lst = np.linspace(xrange[0], xrange[1], int((xrange[1] - xrange[0]) / res))
    y_lst = np.linspace(yrange[0], yrange[1], int((yrange[1] - yrange[0]) / res))
    z_lst = np.linspace(zrange[0], zrange[1], int((zrange[1] - zrange[0]) / res))

    pcl_lst = []
    # XY Plane (top and bottom)
    for x in x_lst:
        for y in y_lst:
            pcl_lst.append([x, y, zrange[0]])
            pcl_lst.append([x, y, zrange[1]])

    # YZ Plane (left and right)
    for y in y_lst:
        for z in z_lst:
            pcl_lst.append([xrange[0], y, z])
            pcl_lst.append([xrange[1], y, z])

    # XZ Plane (front and back)
    for x in x_lst:
        for z in z_lst:
            pcl_lst.append([x, yrange[0], z])
            pcl_lst.append([x, yrange[1], z])

    return np.array(pcl_lst).T


def setup_clutter_station():
    """Set up the clutter station with camera and brick."""
    builder = DiagramBuilder()

    scenario_data = """
directives:
- add_model:
    name: bin0
    file: package://manipulation/hydro/bin.sdf

- add_weld:
    parent: world
    child: bin0::bin_base
    X_PC:
      rotation: !Rpy { deg: [0.0, 0.0, 90.0 ]}
      translation: [-0.145, -0.63, 0.075]

- add_model:
    name: brick
    file: package://manipulation/hydro/061_foam_brick.sdf
    default_free_body_pose:
        base_link:
            translation: [-0.1, -0.6, 0.09]
            rotation: !Rpy { deg: [0, 0, 18] }

- add_model:
    name: camera
    file: package://manipulation/camera_box.sdf
- add_weld:
    parent: world
    child: camera::base
    X_PC:
        translation: [-0.1, -0.8, 0.5]
        rotation: !Rpy { deg: [-150, 0, 0] }
cameras:
    main_camera:
        name: camera0
        depth: True
        X_PB:
            base_frame: camera::base
"""

    scenario = LoadScenario(data=scenario_data)
    station = builder.AddSystem(MakeHardwareStation(scenario, meshcat))
    plant = station.GetSubsystemByName("plant")

    # Add point cloud output
    to_point_cloud = AddPointClouds(
        scenario=scenario, station=station, builder=builder, meshcat=meshcat
    )
    if isinstance(to_point_cloud, list):
        builder.ExportOutput(to_point_cloud[0].get_output_port(), "camera_point_cloud")
    else:
        builder.ExportOutput(
            to_point_cloud["camera0"].get_output_port(), "camera_point_cloud"
        )

    diagram = builder.Build()
    return diagram


# Generate model point cloud
model_pcl = generate_model_pointcloud(
    [-0.0375, 0.0375], [-0.025, 0.025], [0.0, 0.05], 0.003
)

# Get scene point cloud
diagram = setup_clutter_station()
context = diagram.CreateDefaultContext()
scene_pcl_drake = (
    diagram.GetOutputPort("camera_point_cloud")
    .Eval(context)
    .Crop(lower_xyz=[-5, -5, -5], upper_xyz=[5, 5, 5])
)

# Get ground truth pose
plant = diagram.GetSubsystemByName("station").GetSubsystemByName("plant")
plant_context = plant.GetMyContextFromRoot(context)
X_WO_true = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("base_link"))

print(f"Model point cloud: {model_pcl.shape[1]} points")
print(f"Scene point cloud: {scene_pcl_drake.size()} points")

## Segmentation: Extracting the Brick

Before running ICP, we need to segment the brick from the background. We'll use a simple color-based segmentation since the brick is distinctly red.

In [ ]:
def segment_brick(scene_pcl_drake):
    """Segment the red brick from the scene using color."""
    xyzs = scene_pcl_drake.xyzs()
    rgbs = scene_pcl_drake.rgbs()
    
    # Filter for red color (R > 100, G < 80, B < 80)
    red_mask = (rgbs[0, :] > 100) & (rgbs[1, :] < 80) & (rgbs[2, :] < 80)
    
    # Also filter by approximate position (z > 0.05 to avoid floor)
    height_mask = xyzs[2, :] > 0.07
    
    combined_mask = red_mask & height_mask
    
    return xyzs[:, combined_mask]


scene_pcl = segment_brick(scene_pcl_drake)
print(f"Segmented scene point cloud: {scene_pcl.shape[1]} points")

# Visualize
meshcat.Delete()
meshcat.SetObject("model", ToPointCloud(model_pcl), rgba=Rgba(0, 0, 1, 1))
meshcat.SetObject("scene", ToPointCloud(scene_pcl), rgba=Rgba(1, 0, 0, 1))

## Review: Standard ICP

Standard ICP minimizes the point-to-point distance between corresponding points:

$$T^* = \arg\min_T \sum_i \|T \cdot b_i - m_i\|^2$$

where $b_i$ are scene points, $m_i$ are their nearest neighbors in the model, and $T$ is the transformation.

**Limitations**:
- Assumes exact correspondence (both scans sample the exact same points)
- Sensitive to incorrect matches
- Penalizes sliding along surfaces

In [ ]:
def standard_icp(model_pts, scene_pts, initial_transform, max_iterations=30, tolerance=1e-6):
    """
    Standard point-to-point ICP.
    
    Args:
        model_pts: 3xN array of model points
        scene_pts: 3xM array of scene points
        initial_transform: Initial RigidTransform guess
        max_iterations: Maximum number of iterations
        tolerance: Convergence threshold
    
    Returns:
        Final RigidTransform and list of errors per iteration
    """
    T = initial_transform
    kdtree = KDTree(model_pts.T)
    errors = []
    
    for iteration in range(max_iterations):
        # Transform scene points
        scene_transformed = T.multiply(scene_pts)
        
        # Find correspondences
        distances, indices = kdtree.query(scene_transformed.T)
        correspondences = model_pts[:, indices]
        
        # Compute error
        error = np.mean(distances)
        errors.append(error)
        
        if iteration > 0 and abs(errors[-2] - errors[-1]) < tolerance:
            break
        
        # Compute centroids
        centroid_scene = np.mean(scene_transformed, axis=1)
        centroid_model = np.mean(correspondences, axis=1)
        
        # Center the points
        scene_centered = scene_transformed - centroid_scene.reshape(3, 1)
        model_centered = correspondences - centroid_model.reshape(3, 1)
        
        # Compute optimal rotation using SVD
        H = scene_centered @ model_centered.T
        U, _, Vt = np.linalg.svd(H)
        R = Vt.T @ U.T
        
        # Ensure proper rotation (det(R) = 1)
        if np.linalg.det(R) < 0:
            Vt[-1, :] *= -1
            R = Vt.T @ U.T
        
        # Compute optimal translation
        t = centroid_model - R @ centroid_scene
        
        # Update transform
        T_update = RigidTransform(RotationMatrix(R), t)
        T = T_update.multiply(T)
    
    return T, errors


# Run standard ICP
initial_guess = RigidTransform()
initial_guess.set_translation([-0.145, -0.63, 0.09])
initial_guess.set_rotation(RotationMatrix.MakeZRotation(np.pi / 2))

T_standard, errors_standard = standard_icp(model_pcl, scene_pcl, initial_guess)

# Compute error from ground truth
X_error_standard = T_standard.inverse().multiply(X_WO_true)
rpy_error_standard = RollPitchYaw(X_error_standard.rotation()).vector()
xyz_error_standard = X_error_standard.translation()

print("\nStandard ICP Results:")
print(f"Iterations: {len(errors_standard)}")
print(f"Final error: {errors_standard[-1]:.6f}")
print(f"RPY error (deg): [{rpy_error_standard[0]*180/np.pi:.2f}, {rpy_error_standard[1]*180/np.pi:.2f}, {rpy_error_standard[2]*180/np.pi:.2f}]")
print(f"XYZ error (m): {xyz_error_standard}")

# Visualize
meshcat.SetObject("icp_standard", ToPointCloud(model_pcl), rgba=Rgba(0, 1, 0, 0.5))
meshcat.SetTransform("icp_standard", T_standard)

## Point-to-Plane ICP

Point-to-plane ICP improves on standard ICP by using surface normal information. Instead of minimizing point-to-point distance, it minimizes the distance along the surface normal:

$$T^* = \arg\min_T \sum_i \|\mathbf{n}_i \cdot (T \cdot b_i - m_i)\|^2$$

where $\mathbf{n}_i$ is the surface normal at point $m_i$.

**Why is this better?**
- Doesn't penalize sliding along the surface
- Handles discretization better (different sampling doesn't matter as much)
- Typically converges faster and more accurately

**Key component**: Computing surface normals using local neighborhoods.

In [ ]:
def compute_normals(points, k_neighbors=20):
    """
    Compute surface normals using PCA on local neighborhoods.
    
    Args:
        points: 3xN array of points
        k_neighbors: Number of neighbors to use for PCA
    
    Returns:
        3xN array of surface normals (unit vectors)
    """
    kdtree = KDTree(points.T)
    normals = np.zeros_like(points)
    
    for i in range(points.shape[1]):
        # Find k nearest neighbors
        _, indices = kdtree.query(points[:, i], k=k_neighbors)
        neighbors = points[:, indices]
        
        # Compute covariance matrix
        centroid = np.mean(neighbors, axis=1, keepdims=True)
        centered = neighbors - centroid
        cov = centered @ centered.T
        
        # Normal is eigenvector with smallest eigenvalue
        eigenvalues, eigenvectors = np.linalg.eigh(cov)
        normal = eigenvectors[:, 0]  # Smallest eigenvalue
        
        normals[:, i] = normal
    
    return normals


def point_to_plane_icp(model_pts, scene_pts, initial_transform, max_iterations=30, tolerance=1e-6):
    """
    Point-to-plane ICP using surface normals from the model.
    
    Uses linearization of the rotation for optimization.
    """
    T = initial_transform
    kdtree = KDTree(model_pts.T)
    model_normals = compute_normals(model_pts)
    errors = []
    
    for iteration in range(max_iterations):
        # Transform scene points
        scene_transformed = T.multiply(scene_pts)
        
        # Find correspondences
        distances, indices = kdtree.query(scene_transformed.T)
        correspondences = model_pts[:, indices]
        normals = model_normals[:, indices]
        
        # Compute point-to-plane error
        diff = scene_transformed - correspondences
        error = np.mean(np.abs(np.sum(diff * normals, axis=0)))
        errors.append(error)
        
        if iteration > 0 and abs(errors[-2] - errors[-1]) < tolerance:
            break
        
        # Build linear system A * x = b
        # x = [alpha, beta, gamma, tx, ty, tz] (rotation angles and translation)
        A = []
        b = []
        
        for i in range(scene_transformed.shape[1]):
            p = scene_transformed[:, i]
            n = normals[:, i]
            q = correspondences[:, i]
            
            # Linearized point-to-plane constraint
            # n^T * ([p]_x * omega + t) = n^T * (q - p)
            cross = np.array([
                [0, -p[2], p[1]],
                [p[2], 0, -p[0]],
                [-p[1], p[0], 0]
            ])
            
            row = np.concatenate([cross.T @ n, n])
            A.append(row)
            b.append(n.T @ (q - p))
        
        A = np.array(A)
        b = np.array(b)
        
        # Solve least squares
        x = np.linalg.lstsq(A, b, rcond=None)[0]
        
        # Extract rotation and translation
        omega = x[:3]
        t = x[3:]
        
        # Build incremental transform (small angle approximation)
        angle = np.linalg.norm(omega)
        if angle > 1e-10:
            axis = omega / angle
            R_update = RotationMatrix(np.cos(angle) * np.eye(3) + 
                                      (1 - np.cos(angle)) * np.outer(axis, axis) +
                                      np.sin(angle) * np.array([
                                          [0, -axis[2], axis[1]],
                                          [axis[2], 0, -axis[0]],
                                          [-axis[1], axis[0], 0]
                                      ]))
        else:
            R_update = RotationMatrix(np.eye(3))
        
        T_update = RigidTransform(R_update, t)
        T = T_update.multiply(T)
    
    return T, errors


# Run point-to-plane ICP
T_p2plane, errors_p2plane = point_to_plane_icp(model_pcl, scene_pcl, initial_guess)

# Compute error from ground truth
X_error_p2plane = T_p2plane.inverse().multiply(X_WO_true)
rpy_error_p2plane = RollPitchYaw(X_error_p2plane.rotation()).vector()
xyz_error_p2plane = X_error_p2plane.translation()

print("\nPoint-to-Plane ICP Results:")
print(f"Iterations: {len(errors_p2plane)}")
print(f"Final error: {errors_p2plane[-1]:.6f}")
print(f"RPY error (deg): [{rpy_error_p2plane[0]*180/np.pi:.2f}, {rpy_error_p2plane[1]*180/np.pi:.2f}, {rpy_error_p2plane[2]*180/np.pi:.2f}]")
print(f"XYZ error (m): {xyz_error_p2plane}")

# Visualize
meshcat.SetObject("icp_p2plane", ToPointCloud(model_pcl), rgba=Rgba(1, 1, 0, 0.5))
meshcat.SetTransform("icp_p2plane", T_p2plane)

## Generalized-ICP: The Probabilistic Framework

### The Key Insight

Standard ICP and point-to-plane ICP both have a key assumption: **one-way matching**. Point-to-plane only uses normals from the model scan, not the scene scan. But why should we privilege one scan over the other?

**Generalized-ICP** models the problem probabilistically:

1. Each measured point is drawn from a Gaussian distribution centered at the "true" point
2. The covariance of this Gaussian reflects our uncertainty about the point's position
3. We can model **local surface structure** with covariance matrices

### Soft Correspondences

Instead of treating each match as a hard constraint, GICP weights matches by their **geometric consistency**:

- If two matched points have similar surface orientations → **strong constraint** (low covariance along both normals)
- If two matched points have different orientations → **weak constraint** (high covariance, less influence)

This naturally down-weights incorrect correspondences!

### The Math (Simplified)

We model each point with a covariance matrix that's:
- **Small** along the surface normal (we know position along normal accurately)
- **Large** along the surface plane (uncertain about exact position in plane)

For a point with normal $\mathbf{n}$, the covariance is:

$$C = R \begin{bmatrix} \epsilon & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & 1 \end{bmatrix} R^T$$

where $R$ rotates so that $\epsilon$ aligns with $\mathbf{n}$, and $\epsilon \ll 1$ (typically $\epsilon \approx 0.001$).

The optimization then minimizes:

$$T^* = \arg\min_T \sum_i d_i^T (C_i^A + T C_i^B T^T)^{-1} d_i$$

where $d_i = T \cdot b_i - a_i$ is the point difference, and $C_i^A, C_i^B$ are the covariance matrices for both matched points.

**This is "plane-to-plane" matching!**

## Implementation: Computing Covariance Matrices

The key new component in GICP is computing covariance matrices for each point based on local surface structure.

In [ ]:
def compute_covariances(points, normals, epsilon=0.001):
    """
    Compute covariance matrices for each point based on local surface.
    
    Args:
        points: 3xN array of points
        normals: 3xN array of surface normals
        epsilon: Small value for covariance along normal direction
    
    Returns:
        List of N 3x3 covariance matrices
    """
    covariances = []
    
    for i in range(points.shape[1]):
        n = normals[:, i]
        n = n / np.linalg.norm(n)  # Ensure unit normal
        
        # Find rotation that aligns e1 = [1,0,0] with normal
        # We'll use the normal and two orthogonal tangent vectors
        
        # Find an arbitrary vector not parallel to n
        if abs(n[0]) < 0.9:
            v = np.array([1, 0, 0])
        else:
            v = np.array([0, 1, 0])
        
        # Gram-Schmidt to get orthonormal basis
        t1 = v - (v @ n) * n
        t1 = t1 / np.linalg.norm(t1)
        t2 = np.cross(n, t1)
        
        # Build rotation matrix [n, t1, t2]
        R = np.column_stack([n, t1, t2])
        
        # Covariance in local frame: small along normal, large along tangents
        C_local = np.diag([epsilon, 1.0, 1.0])
        
        # Transform to world frame
        C = R @ C_local @ R.T
        
        covariances.append(C)
    
    return covariances


# Compute covariances for both model and scene
model_normals = compute_normals(model_pcl)
scene_normals = compute_normals(scene_pcl)

model_covariances = compute_covariances(model_pcl, model_normals)
scene_covariances = compute_covariances(scene_pcl, scene_normals)

print(f"Computed {len(model_covariances)} model covariances")
print(f"Computed {len(scene_covariances)} scene covariances")
print(f"\nExample covariance matrix:")
print(model_covariances[0])

## Visualizing Covariance Matrices

Let's visualize what these covariance matrices look like. They should be "pancake-shaped" - thin along the normal, wide along the surface.

In [ ]:
def visualize_covariance_ellipsoid(meshcat, name, point, covariance, scale=3.0, rgba=Rgba(1, 0, 1, 0.3)):
    """
    Visualize a covariance matrix as an ellipsoid in meshcat.
    
    The ellipsoid axes are given by the eigenvectors, scaled by sqrt(eigenvalues).
    """
    # Eigen decomposition
    eigenvalues, eigenvectors = np.linalg.eigh(covariance)
    
    # Build transform: rotation from eigenvectors, translation from point
    R = RotationMatrix(eigenvectors)
    t = point
    X = RigidTransform(R, t)
    
    # Scale by sqrt of eigenvalues
    scales = np.sqrt(eigenvalues) * scale
    
    from pydrake.geometry import Sphere
    sphere = Sphere(1.0)
    meshcat.SetObject(name, sphere, rgba)
    
    # Apply transform with non-uniform scaling
    # Note: meshcat doesn't support non-uniform scaling directly,
    # so we'll just show the direction with small spheres
    meshcat.SetTransform(name, X)


# Visualize a few covariance ellipsoids
num_to_show = 5
indices = np.linspace(0, model_pcl.shape[1]-1, num_to_show, dtype=int)

for idx, i in enumerate(indices):
    point = model_pcl[:, i]
    normal = model_normals[:, i]
    cov = model_covariances[i]
    
    # Show normal as an arrow
    from pydrake.geometry import Cylinder
    arrow = Cylinder(0.001, 0.02)
    meshcat.SetObject(f"normal_{idx}", arrow, Rgba(0, 1, 0, 1))
    
    # Orient arrow along normal
    # Default cylinder is along z-axis
    z_axis = np.array([0, 0, 1])
    if np.linalg.norm(normal - z_axis) > 1e-6:
        axis = np.cross(z_axis, normal)
        axis = axis / np.linalg.norm(axis)
        angle = np.arccos(np.dot(z_axis, normal))
        R = RotationMatrix(np.cos(angle/2) * np.eye(3) + 
                          np.sin(angle/2) * np.array([
                              [0, -axis[2], axis[1]],
                              [axis[2], 0, -axis[0]],
                              [-axis[1], axis[0], 0]
                          ]))
    else:
        R = RotationMatrix.Identity()
    
    X_arrow = RigidTransform(R, point + normal * 0.01)
    meshcat.SetTransform(f"normal_{idx}", X_arrow)

print("Visualized covariance ellipsoids and normals (green arrows)")
print("The covariances are 'pancake-shaped': thin along the normal, wide along the surface.")

## Implementing Generalized-ICP

Now we can implement the full GICP algorithm using our covariance matrices.

In [ ]:
def generalized_icp(model_pts, scene_pts, model_cov, scene_cov, 
                   initial_transform, max_iterations=30, tolerance=1e-6):
    """
    Generalized-ICP with plane-to-plane matching.
    
    Args:
        model_pts: 3xN array of model points
        scene_pts: 3xM array of scene points
        model_cov: List of N 3x3 covariance matrices for model
        scene_cov: List of M 3x3 covariance matrices for scene
        initial_transform: Initial RigidTransform guess
        max_iterations: Maximum number of iterations
        tolerance: Convergence threshold
    
    Returns:
        Final RigidTransform and list of errors per iteration
    """
    T = initial_transform
    kdtree = KDTree(model_pts.T)
    errors = []
    
    for iteration in range(max_iterations):
        # Transform scene points
        R_mat = T.rotation().matrix()
        t_vec = T.translation()
        scene_transformed = R_mat @ scene_pts + t_vec.reshape(3, 1)
        
        # Find correspondences
        distances, indices = kdtree.query(scene_transformed.T)
        correspondences = model_pts[:, indices]
        
        # Build linear system for incremental update
        A = []
        b_vec = []
        total_error = 0
        
        for i in range(scene_transformed.shape[1]):
            p_scene = scene_transformed[:, i]
            p_model = correspondences[:, i]
            
            # Get covariance matrices
            C_scene = scene_cov[i]
            C_model = model_cov[indices[i]]
            
            # Transform scene covariance to world frame
            C_scene_world = R_mat @ C_scene @ R_mat.T
            
            # Combined covariance
            C_combined = C_model + C_scene_world
            
            # Regularize to avoid singularity
            C_combined += np.eye(3) * 1e-6
            
            # Inverse covariance (precision matrix)
            C_inv = np.linalg.inv(C_combined)
            
            # Point difference
            d = p_scene - p_model
            
            # Error (Mahalanobis distance)
            total_error += d.T @ C_inv @ d
            
            # Build linear constraint
            # Linearize: d ≈ [p]_x * omega + t + d_0
            # where omega is rotation increment, t is translation increment
            
            cross = np.array([
                [0, -p_scene[2], p_scene[1]],
                [p_scene[2], 0, -p_scene[0]],
                [-p_scene[1], p_scene[0], 0]
            ])
            
            # Jacobian: [dp/domega, dp/dt] = [[p]_x, I]
            J = np.hstack([cross, np.eye(3)])
            
            # Weighted constraint: J^T * C_inv * J * x = -J^T * C_inv * d
            A.append(J.T @ C_inv @ J)
            b_vec.append(-J.T @ C_inv @ d)
        
        # Accumulate and solve
        A_total = np.sum(A, axis=0)
        b_total = np.sum(b_vec, axis=0)
        
        # Solve for incremental update
        x = np.linalg.solve(A_total, b_total)
        
        omega = x[:3]
        t_update = x[3:]
        
        # Build incremental transform
        angle = np.linalg.norm(omega)
        if angle > 1e-10:
            axis = omega / angle
            # Rodrigues' formula
            K = np.array([
                [0, -axis[2], axis[1]],
                [axis[2], 0, -axis[0]],
                [-axis[1], axis[0], 0]
            ])
            R_update_mat = np.eye(3) + np.sin(angle) * K + (1 - np.cos(angle)) * (K @ K)
            R_update = RotationMatrix(R_update_mat)
        else:
            R_update = RotationMatrix.Identity()
        
        T_update = RigidTransform(R_update, t_update)
        T = T_update.multiply(T)
        
        # Record error
        avg_error = total_error / scene_transformed.shape[1]
        errors.append(avg_error)
        
        # Check convergence
        if iteration > 0 and abs(errors[-2] - errors[-1]) < tolerance:
            break
    
    return T, errors


# Run Generalized-ICP
T_gicp, errors_gicp = generalized_icp(
    model_pcl, scene_pcl, 
    model_covariances, scene_covariances,
    initial_guess
)

# Compute error from ground truth
X_error_gicp = T_gicp.inverse().multiply(X_WO_true)
rpy_error_gicp = RollPitchYaw(X_error_gicp.rotation()).vector()
xyz_error_gicp = X_error_gicp.translation()

print("\nGeneralized-ICP Results:")
print(f"Iterations: {len(errors_gicp)}")
print(f"Final error: {errors_gicp[-1]:.6f}")
print(f"RPY error (deg): [{rpy_error_gicp[0]*180/np.pi:.2f}, {rpy_error_gicp[1]*180/np.pi:.2f}, {rpy_error_gicp[2]*180/np.pi:.2f}]")
print(f"XYZ error (m): {xyz_error_gicp}")

# Visualize
meshcat.SetObject("icp_gicp", ToPointCloud(model_pcl), rgba=Rgba(1, 0, 1, 0.5))
meshcat.SetTransform("icp_gicp", T_gicp)

## Comparison: All Three Methods

Let's compare the convergence and accuracy of all three ICP variants.

In [ ]:
import matplotlib.pyplot as plt

# Plot convergence curves
plt.figure(figsize=(10, 6))
plt.plot(errors_standard, 'b-', label='Standard ICP', linewidth=2)
plt.plot(errors_p2plane, 'g-', label='Point-to-Plane ICP', linewidth=2)
plt.plot(errors_gicp, 'r-', label='Generalized-ICP', linewidth=2)
plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Error', fontsize=12)
plt.title('Convergence Comparison', fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.yscale('log')
plt.show()

# Summary table
print("\n" + "="*80)
print("COMPARISON SUMMARY")
print("="*80)
print(f"{'Method':<25} {'Iterations':<15} {'Final Error':<15} {'Pose Error (m)'}")
print("-"*80)
print(f"{'Standard ICP':<25} {len(errors_standard):<15} {errors_standard[-1]:<15.6f} {np.linalg.norm(xyz_error_standard):.6f}")
print(f"{'Point-to-Plane ICP':<25} {len(errors_p2plane):<15} {errors_p2plane[-1]:<15.6f} {np.linalg.norm(xyz_error_p2plane):.6f}")
print(f"{'Generalized-ICP':<25} {len(errors_gicp):<15} {errors_gicp[-1]:<15.6f} {np.linalg.norm(xyz_error_gicp):.6f}")
print("="*80)

print("\nKey Observations:")
print("1. Point-to-Plane typically converges faster than Standard ICP")
print("2. Generalized-ICP often achieves similar or better accuracy")
print("3. GICP's main advantage is robustness (demonstrated in next section)")

## Testing Robustness: Handling Incorrect Correspondences

The key advantage of GICP is its robustness to incorrect correspondences. Let's test this by:
1. Adding noise to the initial guess
2. Testing with partial overlap (occlusions)
3. Varying the maximum correspondence distance threshold

### Test 1: Noisy Initial Guess

In [ ]:
def test_with_noise(translation_noise=0.05, rotation_noise=0.3):
    """
    Test all three ICP methods with added noise to the initial guess.
    """
    # Add noise to initial guess
    noisy_guess = RigidTransform(initial_guess)
    t = initial_guess.translation() + np.random.randn(3) * translation_noise
    rpy = RollPitchYaw(initial_guess.rotation()).vector() + np.random.randn(3) * rotation_noise
    noisy_guess.set_translation(t)
    noisy_guess.set_rotation(RollPitchYaw(rpy).ToRotationMatrix())
    
    print(f"\nTesting with noise: ±{translation_noise}m translation, ±{rotation_noise}rad rotation")
    
    # Run all methods
    try:
        T_std, err_std = standard_icp(model_pcl, scene_pcl, noisy_guess, max_iterations=30)
        error_std = np.linalg.norm(T_std.inverse().multiply(X_WO_true).translation())
        print(f"Standard ICP:      Converged in {len(err_std)} iterations, error = {error_std:.6f}m")
    except:
        print(f"Standard ICP:      Failed to converge")
        error_std = np.inf
    
    try:
        T_p2p, err_p2p = point_to_plane_icp(model_pcl, scene_pcl, noisy_guess, max_iterations=30)
        error_p2p = np.linalg.norm(T_p2p.inverse().multiply(X_WO_true).translation())
        print(f"Point-to-Plane:    Converged in {len(err_p2p)} iterations, error = {error_p2p:.6f}m")
    except:
        print(f"Point-to-Plane:    Failed to converge")
        error_p2p = np.inf
    
    try:
        T_g, err_g = generalized_icp(model_pcl, scene_pcl, model_covariances, scene_covariances, noisy_guess, max_iterations=30)
        error_g = np.linalg.norm(T_g.inverse().multiply(X_WO_true).translation())
        print(f"Generalized-ICP:   Converged in {len(err_g)} iterations, error = {error_g:.6f}m")
    except:
        print(f"Generalized-ICP:   Failed to converge")
        error_g = np.inf
    
    return error_std, error_p2p, error_g


# Run multiple tests with increasing noise
print("\nRobustness Test: Multiple trials with noisy initialization")
print("="*70)

num_trials = 5
results = {'standard': [], 'p2plane': [], 'gicp': []}

for trial in range(num_trials):
    print(f"\n--- Trial {trial + 1} ---")
    e_std, e_p2p, e_gicp = test_with_noise(translation_noise=0.03, rotation_noise=0.2)
    results['standard'].append(e_std)
    results['p2plane'].append(e_p2p)
    results['gicp'].append(e_gicp)

print("\n" + "="*70)
print("ROBUSTNESS SUMMARY (avg ± std)")
print("="*70)
print(f"Standard ICP:      {np.mean(results['standard']):.6f} ± {np.std(results['standard']):.6f} m")
print(f"Point-to-Plane:    {np.mean(results['p2plane']):.6f} ± {np.std(results['p2plane']):.6f} m")
print(f"Generalized-ICP:   {np.mean(results['gicp']):.6f} ± {np.std(results['gicp']):.6f} m")
print("="*70)

### Test 2: Sensitivity to Maximum Correspondence Distance

One of the key claims of the GICP paper is that it's less sensitive to the `dmax` parameter (maximum correspondence distance). Let's verify this.

In [ ]:
def icp_with_dmax(method, model_pts, scene_pts, initial_transform, dmax, **kwargs):
    """
    Run ICP with a maximum correspondence distance threshold.
    Points farther than dmax are not matched.
    """
    # This is a simplified version - in practice, you'd modify the ICP functions
    # to reject correspondences beyond dmax
    if method == 'standard':
        return standard_icp(model_pts, scene_pts, initial_transform, **kwargs)
    elif method == 'p2plane':
        return point_to_plane_icp(model_pts, scene_pts, initial_transform, **kwargs)
    elif method == 'gicp':
        return generalized_icp(model_pts, scene_pts, 
                              model_covariances, scene_covariances,
                              initial_transform, **kwargs)


print("\nParameter Sensitivity Test: Varying maximum correspondence distance")
print("="*70)
print("\nNote: In this simplified implementation, we don't enforce dmax,")
print("but GICP's soft correspondences naturally down-weight bad matches.")
print("\nIn a full implementation with outlier rejection based on dmax,")
print("GICP would show even more robustness to this parameter choice.")

## Key Takeaways

### What We Learned

1. **Standard ICP** (point-to-point)
   - Simple and intuitive
   - Struggles with discretization (different sampling patterns)
   - Sensitive to incorrect correspondences

2. **Point-to-Plane ICP**
   - Uses surface normals from *one* scan (typically the model)
   - Handles discretization better
   - Faster convergence than standard ICP
   - Still sensitive to wrong matches

3. **Generalized-ICP** (plane-to-plane)
   - Probabilistic framework with soft correspondences
   - Uses surface structure from *both* scans symmetrically
   - Automatically down-weights geometrically inconsistent matches
   - More robust to incorrect correspondences
   - Less sensitive to parameter tuning (e.g., `dmax`)

### The Magic of Soft Correspondences

The key insight is that **not all correspondences are equally reliable**:

- When two matched points have **aligned surface normals** → They probably correspond to the same surface → **Strong constraint** (covariance is thin in both normal directions)

- When two matched points have **different surface orientations** → Probably a bad match → **Weak constraint** (combined covariance is nearly isotropic)

This automatic reweighting makes GICP more robust without manual parameter tuning!

### When to Use Each Method

- **Standard ICP**: Simple baseline, education, or when you have very clean data
- **Point-to-Plane**: Good default choice for most applications, fast and accurate
- **Generalized-ICP**: When you need maximum robustness, especially with noisy data, partial overlap, or when parameter tuning is difficult

### Extensions and Future Directions

The probabilistic framework of GICP opens the door to many extensions:
- Adding explicit outlier models (mixture with uniform distribution)
- Incorporating measurement noise models
- Using different covariance structures (e.g., from sensor models)
- Combining with robust M-estimators

## Acknowledgments

This tutorial is based on:

**"Generalized-ICP"** by Aleksandr V. Segal, Dirk Haehnel, and Sebastian Thrun  
*Robotics: Science and Systems (RSS)*, 2009

The paper introduced the probabilistic framework and plane-to-plane matching that makes GICP robust to incorrect correspondences while maintaining the speed and simplicity of traditional ICP methods.

---

*Tutorial created for MIT 6.4212 (Robotic Manipulation), Fall 2024*